In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
from enum import IntEnum
import sys
sys.path.append("./utils")

from generate_features import *
from generate_features2 import *
from get_targets import *
from dnn_utils import *

from stockstats import StockDataFrame

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns

import optuna

Current device: cuda


In [2]:
# def plot_scores(scores, title):
#     plt.figure(figsize=(15, 6))
#     labels = ['Без эмбэддингов', 'Core Web Sm', 'L&Mc', 
#               'Affective Space', 'Affective Space + индикаторы', 'Roberta Base + индикаторы']
#     plt.title(title)
#     sns.barplot(labels, scores)
#     plt.xticks(rotation=45)

In [3]:
best_params = {'time_lag': 50,
 'window': 26,
 'factor': 10.213753830290996,
 'ewm_periods': 25,
 'ewm_alpha': 0.35355971168267786
}

In [4]:
TEST_RATIO = 0.2
EWM_PERIODS = best_params["ewm_periods"]
EWM_ALPHA = best_params["ewm_alpha"]

TIME_LAG = best_params["time_lag"]
WINDOW = best_params["window"]
FACTOR = best_params["factor"]

In [5]:
# ROBERTA_BASE_EMBEDDING_PATH = 'roberta_base_embeddings.csv'
# CORE_WEB_SM_PATH = 'core_web_sm_embeddings.csv'
# AFS_RESULT_PATH = "tesla_dataset_affectivespace.csv"
# LMC_RESULT_PATH = "tesla_dataset_l&mc.csv"

AFS_RESULT_BTC_PATH = 'btc_dataset_small_affectivespace.csv'
ROBERTA_BASE_EMBEDDING_PATH = 'btc_roberta_base.csv'


train_df = pd.read_csv(ROBERTA_BASE_EMBEDDING_PATH, index_col=0)

C:\dev\Anaconda\envs\mipt-stats\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
train_df = indicators_with_rowwise_ewm(train_df)
train_df

MemoryError: Unable to allocate 8.44 GiB for an array with shape (773, 1465998) and data type float64

In [ ]:
targets = get_buy_signal_with_std_threshold_target(train_df, TIME_LAG, WINDOW, FACTOR)
train_df = pd.concat([train_df, targets], axis=1)
train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)


In [ ]:
X = train_df.drop(["target"], axis=1).to_numpy()
y = train_df["target"].to_numpy()

X.shape, y.shape

In [ ]:
train_df['target'].value_counts()

In [ ]:
test_size = int(X.shape[0] * TEST_RATIO)

X_train = X[:-test_size]
y_train = y[:-test_size]
X_test = X[-test_size:]
y_test = y[-test_size:]

train_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train) * np.array([1, 17])
test_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_test), y=y_test)

In [ ]:
lgbm_classifier = LGBMClassifier(class_weight="balanced")
lgbm_classifier.fit(X_train, y_train)

**BTC**

**Affective Space**

`Было`

In [13]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_AFS_btc = roc_auc_score(y_test, probas[:, 1])
accuracy_score_AFS_btc = accuracy_score(y_test, predictions)
print(roc_auc_score_AFS_btc)
print(accuracy_score_AFS_btc)
confusion_matrix(y_test, predictions)

0.6611467236467237
0.9684241019698725


array([[3343,    5],
       [ 104,    0]], dtype=int64)

`С новыми индикаторами`

In [26]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_AFS_btc_2 = roc_auc_score(y_test, probas[:, 1])
accuracy_score_AFS_btc_2 = accuracy_score(y_test, predictions)
print(roc_auc_score_AFS_btc_2)
print(accuracy_score_AFS_btc_2)
confusion_matrix(y_test, predictions)

0.6281446355216529
0.9738673670387393


array([[7405,   16],
       [ 183,   11]], dtype=int64)

**Roberta Base**

`Было`

In [ ]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_RB_btc = roc_auc_score(y_test, probas[:, 1])
accuracy_score_RB_btc = accuracy_score(y_test, predictions)
print(roc_auc_score_RB_btc)
print(accuracy_score_RB_btc)
confusion_matrix(y_test, predictions)

`С новыми индикаторами`

In [10]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
roc_auc_score_RB_btc_2 = roc_auc_score(y_test, probas[:, 1])
accuracy_score_RB_btc_2 = accuracy_score(y_test, predictions)
print(roc_auc_score_RB_btc_2)
print(accuracy_score_RB_btc_2)
confusion_matrix(y_test, predictions)

0.6281446355216529
0.9738673670387393


array([[7405,   16],
       [ 183,   11]], dtype=int64)